# Linear Modeling

The goal of this lab is to design a class for linear regression, using no classes or functions from Scikit-Learn.

Ensure your class satisfies:
- Includes a class method to fit the model to a Pandas dataframe $X$ and a Pandas series $y$
- Includes a class method to solve for the optimal coefficients
- Includes a class method to make predictions, given a new matrix $\hat{X}$
- Does not invert any matrices explicitly. Instead, solve the normal equations using `np.lingalg.solve`.
- It can be instructed to automatically perform a train-test split and return performance metrics on the test set.
- It can provide metrics including SSE, MSE, RMSE, and $R^2$.

Before programming your class, consider the following questions and record the answers:
- How does your class handle categorical data? How does Sci-kit do it?
- How does your class handle missing data? How does Sci-kit do it?
- Does your class have any methods for creating polynomial expansions or otherwise transforming data? How does Sci-kit do it?
- How does your class handle the bias/intercept/constant? How does Sci-kit do it?
- What output do you automatically provide to the user? Why? How does Sci-kit do it?
- Are you including any tools for statistical inference? How does Sci-kit do it?

In order to measure how long it takes to run code, you can `import time`, and 

```
start = time.time()
<expressions and code>
finish = time.time()
runtime = start-finish
```

For the `heart_hw.csv` and `cars_hw.csv` data in the assignment folder, run some regressions and compare the performance of your class with Sci-Kit's linear regression model. Do you get the same answers for the optimal coefficients, SSE, and $R^2$? Which one runs faster?



In [33]:
import pandas as pd
import numpy as np
import time
class MagicRegression:
    def __init__(self):
        self.coefficients = None
        self.intercept = None
        self.fitted = False
        self.columns = None

    def randomize_data(self, X, y):
        indicies = np.arange(X.shape[0])
        np.random.shuffle(indicies)
        return X[indicies], y[indicies]
    
    def train_test_split(self, X, y, test_size=0.5, randomize=True):
        if randomize == True:
            X, y = self.randomize_data(X, y)
        split = len(y) - int(len(y)*test_size)
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]
        return X_train, X_test, y_train, y_test
        
    def standardize(self, X_train, X_test):
        X_train = X_train.astype(np.float64)
        X_test = X_test.astype(np.float64)
        
        mean = np.mean(X_train, axis=0)
        std = np.std(X_train, axis=0)
        X_train = (X_train - mean) / std
        X_test = (X_test - mean) / std
        return X_train, X_test

    
    def reshape(self, X_train, X_test):
        if len(X_train.shape) == 1:
            X_train = np.expand_dims(X_train, axis=-1)
            X_test = np.expand_dims(X_test, axis=-1)
        return X_train, X_test
    
    def design_matrix(self, X):
        n = len(X)
        X = np.column_stack((np.ones(n), X))
        return X

    def encode_categorical(self, X):
        X_encoded = pd.get_dummies(X, drop_first=True)
        return X_encoded

    def MagicFit(self, X, y, test_size = 0.2):
        X_encoded = self.encode_categorical(X)
        self.columns = X_encoded.columns

        X_train_df, X_test_df, y_train, y_test = self.train_test_split(X_encoded.to_numpy(), y.to_numpy(), test_size=test_size)

        X_train_df, X_test_df = self.standardize(X_train_df, X_test_df)
        X_train_df, X_test_df = self.reshape(X_train_df, X_test_df)

        X_train_design = self.design_matrix(X_train_df)
        X_test_design = self.design_matrix(X_test_df)

        XT_X = X_train_design.T.dot(X_train_design)
        XT_y = X_train_design.T.dot(y_train)
        beta = np.linalg.solve(XT_X, XT_y)

        self.intercept = beta[0]
        self.coefficients = beta[1:]
        self.fitted = True

        y_pred = X_test_design.dot(beta)
        sse = np.sum((y_test - y_pred) ** 2)
        mse = sse / len(y_test)
        rmse = np.sqrt(mse)
        r_squared = 1 - sse / np.sum((y_test - np.mean(y_test)) ** 2)
        print(f'SSE:{sse:.2f}')
        print(f'MSE:{mse:.2f}')
        print(f'RMSE:{rmse:.2f}')
        print(f'R^2:{r_squared:.2f}')
    
    

In [34]:
df = pd.read_csv('../data/cars_hw.csv')
df.dropna(inplace=True)
X = df[['Mileage_Run', 'Body_Type', 'Make']]
y = df['Price']

model = MagicRegression()

start = time.time()
model.MagicFit(X, y)
finish = time.time()
run_time = finish - start 
print('The Linear Regression Model from scratch ran for approximately',round(run_time,6), 'seconds\n')



from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
X_encoded = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2)

model_sklearn = LinearRegression()
model_sklearn.fit(X_train, y_train)

start_sklearn = time.time()
y_pred = model_sklearn.predict(X_test)
finish_sklearn = time.time()
run_time_sklearn = finish_sklearn - start_sklearn

r2_sklearn = r2_score(y_test, y_pred)
mse_sklearn = mean_squared_error(y_test, y_pred)
sse_sklearn = mse_sklearn * len(y_test)
rmse_sklearn = root_mean_squared_error(y_test, y_pred)

print(f'SSE from sklearn:{sse_sklearn:.2f}')
print(f'MSE from sklearn:{mse_sklearn:.2f}')
print(f'RMSE for sklearn:{rmse_sklearn:.2f}')
print(f'R^2 from sklearn:{r2_sklearn:.2f}')
print('The Linear Regression Model From sklearn ran for approximately', round(run_time_sklearn, 6), 'seconds')

SSE:7699505406902.23
MSE:39484643112.32
RMSE:198707.43
R^2:0.67
The Linear Regression Model from scratch ran for approximately 0.00414 seconds

SSE from sklearn:9257123357842.00
MSE from sklearn:47230221213.48
RMSE for sklearn:217325.15
R^2 from sklearn:0.63
The Linear Regression Model From sklearn ran for approximately 0.001086 seconds


Apparently, the algorithm that I coded from scratch performs better, if not much better than the built in package from sklearn. The only difference here is that sklearn tends to run faster, performing the calculation at 0.001086 seconds compared to 0.00414 seconds from my algorithm.

In [71]:
df2 = pd.read_csv('../data/airbnb_hw.csv')
df2.dropna(inplace=True)
X = df2[['Room Type', 'Review Scores Rating (bin)','Review Scores Rating']]
y = df2['Price'].astype(str).str.replace(r'[\$,]', '', regex=True)
y = pd.to_numeric(y, errors='coerce')

model = MagicRegression()

start = time.time()
model.MagicFit(X, y)
finish = time.time()
run_time = finish - start 
print('The Linear Regression Model from scratch ran for approximately',round(run_time,6), 'seconds\n')


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
X_encoded = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2)

model_sklearn = LinearRegression()
model_sklearn.fit(X_train, y_train)

start_sklearn = time.time()
y_pred = model_sklearn.predict(X_test)
finish_sklearn = time.time()
run_time_sklearn = finish_sklearn - start_sklearn

r2_sklearn = r2_score(y_test, y_pred)
mse_sklearn = mean_squared_error(y_test, y_pred)
sse_sklearn = mse_sklearn * len(y_test)
rmse_sklearn = root_mean_squared_error(y_test, y_pred)

print(f'SSE from sklearn:{sse_sklearn:.2f}')
print(f'MSE from sklearn:{mse_sklearn:.2f}')
print(f'RMSE for sklearn:{rmse_sklearn:.2f}')
print(f'R^2 from sklearn:{r2_sklearn:.2f}')
print('The Linear Regression Model From sklearn ran for approximately', round(run_time_sklearn, 6), 'seconds')

SSE:58141611.84
MSE:13208.00
RMSE:114.93
R^2:0.20
The Linear Regression Model from scratch ran for approximately 0.010934 seconds

SSE from sklearn:66882356.31
MSE from sklearn:15193.63
RMSE for sklearn:123.26
R^2 from sklearn:0.18
The Linear Regression Model From sklearn ran for approximately 0.000889 seconds


Since for some reason the heart_hw.csv file couldn't be accessed, I used airbnb_hw.csv's data to perform linear regression. It turns out that this time, my algorithm still does slightly better than sklearn's model, while sometimes it is vice versa as I click 'run' several times. But overall, it is true that sklearn does run faster than the algorithm I created, but at the cost of having a higher sum of squared error and R^2 value. 